In [1]:
import numpy as np
import math
from qutip import*
import matplotlib.pyplot as plt
from qutip.qip.circuit import Gate
from qutip.qip.circuit import QubitCircuit, Gate
from qutip import tensor, basis
import itertools
pi=np.pi
import time

# Noisefree circuit (QuTiP)


In [2]:
I= qeye(2)
X= sigmax()
Y= sigmay()
Z= sigmaz()

import xlrd
loc = ("Cost_Function_LiH")
wb = xlrd.open_workbook(r'C:\Users\debnathk\OneDrive - Chalmers\Postdoc Chalmers\QSimulator_with_noise_two_levels_v3\VQE_phalgun\Cost_Function_LiH.xls')
sheet = wb.sheet_by_index(0)

Cost_function= qeye(16)*0
Cost_function.dims= [[2, 2, 2, 2], [2, 2, 2, 2]]

for Row in range(100):
    oper= []
    for Column in range(4):
        operator= sheet.cell_value(Row, Column)
        if operator == 'I':
            oper.append(I)
        elif operator == 'X':
            oper.append(X)
        elif operator == 'Y':
            oper.append(Y)
        elif operator == 'Z':
            oper.append(Z)
        else:
            print("Error in determining operator")
                
    Cost_function = Cost_function + sheet.cell_value(Row,4)*tensor(oper) 
    
def noiseless(T, *args):
        
    qc = QubitCircuit(4)
    qc.add_gate("RY", targets=[0], arg_value= T[0])
    qc.add_gate("RY", targets=[1], arg_value= T[1])
    qc.add_gate("RY", targets=[2], arg_value= T[2])
    qc.add_gate("RY", targets=[3], arg_value= T[3])
    qc.add_gate("CZ", controls=[0], targets=[1])    
    qc.add_gate("RY", targets=[0], arg_value= T[4])
    qc.add_gate("CZ", controls=[1], targets=[3])
    qc.add_gate("CZ", controls=[1], targets=[2])
    qc.add_gate("RY", targets=[3], arg_value= T[5])
    qc.add_gate("RY", targets=[1], arg_value= T[6])
    qc.add_gate("RY", targets=[2], arg_value= T[7])
    qc.add_gate("CZ", controls=[0], targets=[1])
    qc.add_gate("RY", targets=[0], arg_value= T[8])
    qc.add_gate("CZ", controls=[1], targets=[3])
    qc.add_gate("CZ", controls=[1], targets=[2])
    qc.add_gate("RY", targets=[1], arg_value= T[9])
    qc.add_gate("RY", targets=[2], arg_value= T[10])
    qc.add_gate("RY", targets=[3], arg_value= T[11])

    
    U= qc.propagators()[0]
    state=  basis(2,0)
    initial_state= tensor(state, state, state, state)
    final_state=  qc.run(state=initial_state)
    cfunction= expect(Cost_function, final_state)
    return cfunction


from scipy.optimize import minimize
Energy= []
parameters= []
for i in range(100):
    guess_parameters= np.random.random(12) - 0.5
    res2= minimize(noiseless, guess_parameters, method='Powell',  options={'maxiter':100000})
    Energy.append(res2.fun)
    parameters.append(res2.x)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [3]:
def state_return(T, *args):
        
    qc = QubitCircuit(4)
    qc.add_gate("RY", targets=[0], arg_value= T[0])
    qc.add_gate("RY", targets=[1], arg_value= T[1])
    qc.add_gate("RY", targets=[2], arg_value= T[2])
    qc.add_gate("RY", targets=[3], arg_value= T[3])
    qc.add_gate("CZ", controls=[0], targets=[1])    
    qc.add_gate("RY", targets=[0], arg_value= T[4])
    qc.add_gate("CZ", controls=[1], targets=[3])
    qc.add_gate("CZ", controls=[1], targets=[2])
    qc.add_gate("RY", targets=[3], arg_value= T[5])
    qc.add_gate("RY", targets=[1], arg_value= T[6])
    qc.add_gate("RY", targets=[2], arg_value= T[7])
    qc.add_gate("CZ", controls=[0], targets=[1])
    qc.add_gate("RY", targets=[0], arg_value= T[8])
    qc.add_gate("CZ", controls=[1], targets=[3])
    qc.add_gate("CZ", controls=[1], targets=[2])
    qc.add_gate("RY", targets=[1], arg_value= T[9])
    qc.add_gate("RY", targets=[2], arg_value= T[10])
    qc.add_gate("RY", targets=[3], arg_value= T[11])

    U= qc.propagators()[0]
    state=  basis(2,0)
    initial_state= tensor(state, state, state, state)
    final_state=  qc.run(state=initial_state)
    
    return ket2dm(final_state)   

Energy_noisefree= np.min(Energy)
Index_noisefree= np.where(np.min(Energy)==Energy)
Eigenstate_noisefree= state_return(parameters[Index_noisefree[0][0]])

# In the presence of noise (propagators)

In [5]:
import pickle
with open('Data\PauliX','rb') as read_file:
    PX = pickle.load(read_file)
    
with open('Data\PauliY','rb') as read_file:
    PY = pickle.load(read_file)
    
with open('Data\PauliZ','rb') as read_file:
    PZ = pickle.load(read_file)
    
with open('Data\Idle','rb') as read_file:
    Idle = pickle.load(read_file)
    
with open('Data\CZ01','rb') as read_file:
    CZ01 = pickle.load(read_file)

with open('Data\Time','rb') as read_file:
    t = pickle.load(read_file)
    
with open('Data\CZ12','rb') as read_file:
    CZ12 = pickle.load(read_file)    
    
with open('Data\CZ13','rb') as read_file:
    CZ13 = pickle.load(read_file)
    
gate_times = [2.0e-8, 2.0e-8, 2.0e-8, 2.71e-7]
Omega = [pi/gate_times[0], pi/gate_times[1], pi/gate_times[2], pi/gate_times[3]]



def truncated_remainder(dividend, divisor):
    divided_number = dividend / divisor
    divided_number = \
        -int(-divided_number) if divided_number < 0 else int(divided_number)

    remainder = dividend - divisor * divided_number

    return remainder


def transform_to_pipi(input_angle):
    revolutions = int((input_angle + np.sign(input_angle) * pi) / (2 * pi))

    p1 = truncated_remainder(input_angle + np.sign(input_angle) * pi, 2 * pi)
    p2 = (np.sign(np.sign(input_angle) + 2 * (np.sign(fabs((truncated_remainder(input_angle + pi, 2 * pi))/ (2 * pi))) - 1))) * pi

    output_angle = p1-p2
    
    if output_angle<0:
        output_angle= math.radians(math.degrees(output_angle + 360))
    
    return output_angle


def Minimize_function(T, *args):
    
    T0= math.radians(math.degrees(T[0]) + 360) % (2*pi)
    T1= math.radians(math.degrees(T[1]) + 360) % (2*pi)
    T2= math.radians(math.degrees(T[2]) + 360) % (2*pi)
    T3= math.radians(math.degrees(T[3]) + 360) % (2*pi)
    T4= math.radians(math.degrees(T[4]) + 360) % (2*pi)
    T5= math.radians(math.degrees(T[5]) + 360) % (2*pi)
    T6= math.radians(math.degrees(T[6]) + 360) % (2*pi)
    T7= math.radians(math.degrees(T[7]) + 360) % (2*pi)
    T8= math.radians(math.degrees(T[8]) + 360) % (2*pi)
    T9= math.radians(math.degrees(T[9]) + 360) % (2*pi)
    T10= math.radians(math.degrees(T[10]) + 360) % (2*pi)
    T11= math.radians(math.degrees(T[11]) + 360) % (2*pi)

        
        
    
    # Gate Sequence 1
    max_gate_time= np.max(np.divide([T0, T1, T2, T3], Omega[1]))
    
    Lis= (np.abs(t-(T0/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[0][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_a= res*Idle[0][Ind2] 
    
    Lis= (np.abs(t-(T1/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[1][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_b= res*Idle[1][Ind2]     
    
    
    Lis= (np.abs(t-(T2/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[2][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_c= res*Idle[2][Ind2] 
                
    Lis= (np.abs(t-(T3/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[3][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_d= res*Idle[3][Ind2] 
    
    U1= super_tensor(U1_a[0], U1_b[0], U1_c[0], U1_d[0])

    
    
    
    # Gate Sequence 2

    temp= super_tensor(CZ01, Idle[2][-1],Idle[3][-1])
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]]
    U2= temp
    
    
    
    # Gate Sequence 3
    max_gate_time= gate_times[-1]
    Lis= (np.abs(t-(T4/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[0][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U3_a= res*Idle[0][Ind2]
    
    id= to_super(qeye(2))
    temp= super_tensor(U3_a[0], CZ13)
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]]
    U3= (temp * super_tensor(id, id, Idle[2][-1], id))
    
    
    
    # Gate Sequence 4
    U4_a= Idle[0][-1]
    
    max_gate_time= gate_times[-1]
    Lis= (np.abs(t-(T5/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[3][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U4_c= res*Idle[3][Ind2]    
    temp= super_tensor(U4_a, CZ12, U4_c[0])
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]]
    U4= temp
    
    
    
#     # Gate sequence 5
    max_gate_time= np.max(np.divide([T6, T7], Omega[1]))
    
    Lis= (np.abs(t-max_gate_time))
    Ind= np.where(Lis==np.min(Lis))
    U5_a= Idle[0][Ind]  
    U5_d= Idle[3][Ind]
      
    Lis= (np.abs(t-(T6/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[1][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U5_b= res*Idle[1][Ind2] 

    Lis= (np.abs(t-(T7/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[2][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U5_c= res*Idle[2][Ind2]    
    
    U5= super_tensor(U5_a[0], U5_b[0], U5_c[0], U5_d[0])  
    
    
    
#     # Gate sequence 6
    U6_a= CZ01
    U6_c= Idle[2][-1]
    U6_d= Idle[3][-1]
    temp= super_tensor(U6_a, U6_c, U6_d)
    temp.dims=  [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]] 
    U6= temp
        
    
    
#     # Gate sequence 7
    max_gate_time= gate_times[-1]
    
    Lis= (np.abs(t-(T8/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[0][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U7_a= res*Idle[0][Ind2] 
    U7_b= CZ13
    temp= super_tensor(U7_a[0], U7_b)
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]] 
    U7= temp*super_tensor(id, id, Idle[2][-1], id)
          
        
#     # Gate sequence 8
    max_gate_time= gate_times[-1]
    U8_a= Idle[0][-1]
    
    U8_b= CZ12
    
    Lis= (np.abs(t-(T11/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[3][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U8_d= res*Idle[3][Ind2] 
    temp= super_tensor(U8_a, U8_b, U8_d[0])
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]] 
    U8= temp
    
            

        
#     # Gate sequence 9
    max_gate_time= np.max(np.divide([T9, T10], Omega[1]))
    Lis= (np.abs(t-max_gate_time))
    Ind= np.where(Lis==np.min(Lis))
    U9_a= Idle[0][Ind]     
    U9_d= Idle[3][Ind]     
    
    Lis= (np.abs(t-(T9/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[1][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U9_b= res*Idle[1][Ind2]  
    
    Lis= (np.abs(t-(T10/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[2][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U9_c= res*Idle[2][Ind2]  
    U9= super_tensor(U9_a[0], U9_b[0], U9_c[0], U9_d[0])
    
    initial_state= initial_state= operator_to_vector(ket2dm(tensor(basis(2,0), basis(2,0), basis(2,0), basis(2,0))))
    final_state= U1*initial_state
    
    final_state= Qobj(np.reshape(np.array(final_state),(16,16)))
    final_state.dims= [[2, 2, 2, 2], [2, 2, 2, 2]]
    cfunction= np.real(expect(Cost_function, final_state))
    
    return cfunction



from scipy.optimize import minimize
Energy_noise=[]
parameters=[]
for i in range(100):
    guess_parameters= np.random.random(12) - 0.5
    res2= minimize(Minimize_function, guess_parameters, method='Powell',  options={'maxiter':100000})
    Energy_noise.append(res2.fun)
    parameters.append(res2.x)      
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [12]:
def Minimize_function_state_return(T, *args):
    
    T0= math.radians(math.degrees(T[0]) + 360) % (2*pi)
    T1= math.radians(math.degrees(T[1]) + 360) % (2*pi)
    T2= math.radians(math.degrees(T[2]) + 360) % (2*pi)
    T3= math.radians(math.degrees(T[3]) + 360) % (2*pi)
    T4= math.radians(math.degrees(T[4]) + 360) % (2*pi)
    T5= math.radians(math.degrees(T[5]) + 360) % (2*pi)
    T6= math.radians(math.degrees(T[6]) + 360) % (2*pi)
    T7= math.radians(math.degrees(T[7]) + 360) % (2*pi)
    T8= math.radians(math.degrees(T[8]) + 360) % (2*pi)
    T9= math.radians(math.degrees(T[9]) + 360) % (2*pi)
    T10= math.radians(math.degrees(T[10]) + 360) % (2*pi)
    T11= math.radians(math.degrees(T[11]) + 360) % (2*pi)

        
        
    
    # Gate Sequence 1
    max_gate_time= np.max(np.divide([T0, T1, T2, T3], Omega[1]))
    
    Lis= (np.abs(t-(T0/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[0][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_a= res*Idle[0][Ind2] 
    
    Lis= (np.abs(t-(T1/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[1][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_b= res*Idle[1][Ind2]     
    
    
    Lis= (np.abs(t-(T2/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[2][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_c= res*Idle[2][Ind2] 
                
    Lis= (np.abs(t-(T3/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[3][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U1_d= res*Idle[3][Ind2] 
    
    U1= super_tensor(U1_a[0], U1_b[0], U1_c[0], U1_d[0])

    
    
    
    # Gate Sequence 2
    temp= super_tensor(CZ01, Idle[2][-1],Idle[3][-1])
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]]
    U2= temp
    
    
    
    # Gate Sequence 3
    max_gate_time= gate_times[-1]
    Lis= (np.abs(t-(T4/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[0][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U3_a= res*Idle[0][Ind2]
    
    id= to_super(qeye(2))
    temp= super_tensor(U3_a[0], CZ13)
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]]
    U3= (temp * super_tensor(id, id, Idle[2][-1], id))
    
    
    
    # Gate Sequence 4
    U4_a= Idle[0][-1]
    
    max_gate_time= gate_times[-1]
    Lis= (np.abs(t-(T5/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[3][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U4_c= res*Idle[3][Ind2]    
    temp= super_tensor(U4_a, CZ12, U4_c[0])
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]]
    U4= temp
    
    
    
#     # Gate sequence 5
    max_gate_time= np.max(np.divide([T6, T7], Omega[1]))
    
    Lis= (np.abs(t-max_gate_time))
    Ind= np.where(Lis==np.min(Lis))
    U5_a= Idle[0][Ind]  
    U5_d= Idle[3][Ind]
      
    Lis= (np.abs(t-(T6/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[1][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U5_b= res*Idle[1][Ind2] 

    Lis= (np.abs(t-(T7/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[2][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U5_c= res*Idle[2][Ind2]    
    
    U5= super_tensor(U5_a[0], U5_b[0], U5_c[0], U5_d[0])  
    
    
    
#     # Gate sequence 6
    U6_a= CZ01
    U6_c= Idle[2][-1]
    U6_d= Idle[3][-1]
    temp= super_tensor(U6_a, U6_c, U6_d)
    temp.dims=  [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]] 
    U6= temp
        
    
    
#     # Gate sequence 7
    max_gate_time= gate_times[-1]
    
    Lis= (np.abs(t-(T8/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[0][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U7_a= res*Idle[0][Ind2] 
    U7_b= CZ13
    temp= super_tensor(U7_a[0], U7_b)
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]] 
    U7= temp*super_tensor(id, id, Idle[2][-1], id)
          
        
#     # Gate sequence 8
    max_gate_time= gate_times[-1]
    U8_a= Idle[0][-1]
    
    U8_b= CZ12
    
    Lis= (np.abs(t-(T11/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[3][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U8_d= res*Idle[3][Ind2] 
    temp= super_tensor(U8_a, U8_b, U8_d[0])
    temp.dims= [[[2, 2, 2, 2], [2, 2, 2, 2]], [[2, 2, 2, 2], [2, 2, 2, 2]]] 
    U8= temp
    
            

        
#     # Gate sequence 9
    max_gate_time= np.max(np.divide([T9, T10], Omega[1]))
    Lis= (np.abs(t-max_gate_time))
    Ind= np.where(Lis==np.min(Lis))
    U9_a= Idle[0][Ind]     
    U9_d= Idle[3][Ind]     
    
    Lis= (np.abs(t-(T9/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[1][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U9_b= res*Idle[1][Ind2]  
    
    Lis= (np.abs(t-(T10/Omega[1])))
    Ind= np.where(Lis==np.min(Lis))
    res= PY[2][Ind]                
    time_diff_for_idle= np.abs(t[Ind]-max_gate_time)
    time_list= np.abs(t-time_diff_for_idle)
    Ind2= np.where(time_list==np.min(time_list))
    U9_c= res*Idle[2][Ind2]  
    U9= super_tensor(U9_a[0], U9_b[0], U9_c[0], U9_d[0])
    
    initial_state= operator_to_vector(ket2dm(tensor(basis(2,0), basis(2,0), basis(2,0), basis(2,0))))
    final_state= U1*initial_state
    
    final_state= Qobj(np.reshape(np.array(final_state),(16,16)))
    final_state.dims= [[2, 2, 2, 2], [2, 2, 2, 2]]
    
    return final_state


Energy_noise= np.min(Energy_noise)
Index_noise= np.where(min_noise==Energy_noise)
Eigenstate_noise= Minimize_function_state_return(parameters[Index_noise[0][0]])

In [13]:
Fide= fidelity(Eigenstate_noisefree, Eigenstate_noise)
print("Noisefree ground state energy is ", Energy_noisefree)
print("With noise ground state energy is", Energy_noise)
print("-----------------------------------------------------------")
print("Fidelity is", Fide)

Noisefree ground state energy is  -1.0781463904932822
With noise ground state energy is -1.0580604024404048
-----------------------------------------------------------
Fidelity is 0.986444786356285
